<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# SmolAgents Prompt Chaining Agent

In this tutorial, we'll explore **prompt chaining** with [Smolagents](https://huggingface.co/docs/smolagents/index) and how to trace using Phoenix.
Prompt chaining is a method where a complex task is broken into smaller, linked subtasks, with the output of one step feeding into the next. This workflow is ideal when a task can be cleanly decomposed into fixed subtasks, making each LLM call simpler and more accurate — trading off latency for better overall performance.

In this example, we'll create a multi-step resume analysis, from extracting keywords to summarizing the analysis — all automated and fully traceable.

By the end of this tutorial, you’ll learn how to:

- Set up specialized Smolagents

- Chain tasks across agents with carryover context

- Improve task accuracy by simplifying each LLM call

- Trace and visualize the full execution using Phoenix

⚠️ You'll need an Hugging Face Token for this tutorial.

## Set up Keys and Dependencies


In [ ]:
!pip install openinference-instrumentation-smolagents smolagents

In [ ]:
import os
from getpass import getpass

from smolagents import CodeAgent, HfApiModel, tool

from phoenix.otel import register

if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass("🔑 Enter your Hugging Face Token: ")

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("🔑 Enter your Phoenix API key: ")

if "PHOENIX_COLLECTOR_ENDPOINT" not in os.environ:
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass("🔑 Enter your Phoenix Collector Endpoint")

## Configure Model

In [ ]:
# Initialize the model
model = HfApiModel()

## Configure Tracing


In [ ]:
tracer_provider = register(
    project_name="smolagents-prompt-chaining",
    protocol="http/protobuf",
    auto_instrument=True,
)

## Example Prompt Chaining Task: Resume Analysis

This example demonstrates how to use Smolagents to do prompt chaining for resume analysis. The resume is analyzed for keywords, skills, and experiences and then the analysis is summarized.

## Define Agent

In [ ]:
# Define the extract_keywords tool


@tool
def extract_keywords(text: str) -> str:
    """
    Extracts keywords from the input text.

    Args:
        text: The resume text to extract keywords from.

    Returns:
        A comma-separated string of keywords with more than 5 letters.
    """
    words = text.split()
    keywords = [word.strip(".,") for word in words if len(word) > 5]
    return ", ".join(keywords)


# Create the agent with the custom tool
agent = CodeAgent(tools=[extract_keywords], model=model)

# Run the agent with a prompt that chains reasoning
agent.run(
    "Extract keywords from this resume: 'Experienced software engineer with 5+ years in Python, "
    "React, and cloud technologies. Led development of microservices architecture and implemented "
    "CI/CD pipelines. Strong background in machine learning and data analysis.' "
    "Then summarize what these keywords suggest about the candidate's technical expertise."
)